In [1]:
from alpdesign.utils import *
from alpdesign.mlp import *
from jax_unirep import get_reps
import alpdesign
import numpy as np
import matplotlib.pyplot as plt
import jax_unirep
import haiku as hk
import jax
import jax.numpy as jnp
import functools
from sklearn.decomposition import PCA

In [2]:
AA_list = ['A','R','N','D','C','Q','E','G','H','I','L','K','M','F','P','S','T','W','Y','V','B','Z','X','*']
blosum92 = np.loadtxt("./blosum62.txt", dtype='i', delimiter=' ')
print(blosum92)
print(len(AA_list))

[[ 4 -1 -2 -2  0 -1 -1  0 -2 -1 -1 -1 -1 -2 -1  1  0 -3 -2  0 -2 -1  0 -4]
 [-1  5  0 -2 -3  1  0 -2  0 -3 -2  2 -1 -3 -2 -1 -1 -3 -2 -3 -1  0 -1 -4]
 [-2  0  6  1 -3  0  0  0  1 -3 -3  0 -2 -3 -2  1  0 -4 -2 -3  3  0 -1 -4]
 [-2 -2  1  6 -3  0  2 -1 -1 -3 -4 -1 -3 -3 -1  0 -1 -4 -3 -3  4  1 -1 -4]
 [ 0 -3 -3 -3  9 -3 -4 -3 -3 -1 -1 -3 -1 -2 -3 -1 -1 -2 -2 -1 -3 -3 -2 -4]
 [-1  1  0  0 -3  5  2 -2  0 -3 -2  1  0 -3 -1  0 -1 -2 -1 -2  0  3 -1 -4]
 [-1  0  0  2 -4  2  5 -2  0 -3 -3  1 -2 -3 -1  0 -1 -3 -2 -2  1  4 -1 -4]
 [ 0 -2  0 -1 -3 -2 -2  6 -2 -4 -4 -2 -3 -3 -2  0 -2 -2 -3 -3 -1 -2 -1 -4]
 [-2  0  1 -1 -3  0  0 -2  8 -3 -3 -1 -2 -1 -2 -1 -2 -2  2 -3  0  0 -1 -4]
 [-1 -3 -3 -3 -1 -3 -3 -4 -3  4  2 -3  1  0 -3 -2 -1 -3 -1  3 -3 -3 -1 -4]
 [-1 -2 -3 -4 -1 -2 -3 -4 -3  2  4 -2  2  0 -3 -2 -1 -2 -1  1 -4 -3 -1 -4]
 [-1  2  0 -1 -3  1  1 -2 -1 -3 -2  5 -1 -3 -1  0 -1 -3 -2 -2  0  1 -1 -4]
 [-1 -1 -2 -3 -1  0 -2 -3 -2  1  2 -1  5  0 -2 -1 -1 -1 -1  1 -3 -1 -1 -4]
 [-2 -3 -3 -3 -2 -3 -3 -3

In [3]:
avg92 = jnp.sum(blosum92)/24/24
sum92 = 0.
for row in blosum92:
    for aa in row:
        sum92 += (aa-avg92)**2
std92 = jnp.sqrt(sum92 / 24/24)
print(avg92)
print(std92)

-1.2604167
2.166242


In [4]:
def blosum(seq1, seq2):
    seqlist1 = list(seq1)
    seqlist2 = list(seq2)
    score = 0.
    for i in range(len(seqlist1)):
        idx1 = AA_list.index(seqlist1[i])
        idx2 = AA_list.index(seqlist2[i])
        score += (blosum92[idx1][idx2] - avg92)/std92
        #jax.nn.sigmoid(score/len(seqlist1))
    return score/len(seqlist1)
    #return score/len(seqlist1)

In [5]:
target_seq = 'TARGETPEPTIDE'
oh_vec = encode_seq(list(target_seq))
oh_unirep = seq2useq(oh_vec)
target_rep = differentiable_jax_unirep(oh_unirep)

seqs = ['ISQLRYICEVIWF']

reps = get_reps(seqs)[0]
labels = []
for seq in seqs:
    labels.append(blosum(target_seq, seq))
labels = np.array(labels)

print(labels)

[0.01368617]


In [6]:
key = jax.random.PRNGKey(0)
c = alpdesign.EnsembleBlockConfig()
forward_t, full_forward_t, seq_t = alpdesign.build_e2e(c)
def gen(k, n): return jax.random.normal(key, shape=(n, 13, 20))
'''
key1, key2 = jax.random.split(key)
start_params = seq_t.init(key1, jnp.tile(
    jnp.squeeze(gen(key2, 1)), (c.model_number, 1)))
best_v, params, train_loss, bo_loss = alpdesign.alg_iter(
    key2, reps, labels, full_forward_t, seq_t, c, x0_gen=gen,
    start_params=start_params
)
'''


'\nkey1, key2 = jax.random.split(key)\nstart_params = seq_t.init(key1, jnp.tile(\n    jnp.squeeze(gen(key2, 1)), (c.model_number, 1)))\nbest_v, params, train_loss, bo_loss = alpdesign.alg_iter(\n    key2, reps, labels, full_forward_t, seq_t, c, x0_gen=gen,\n    start_params=start_params\n)\n'

In [7]:
reps = get_reps(seqs)[0]
labels = []
for seq in seqs:
    labels.append(blosum(target_seq, seq))
labels = np.array(labels)
def loop(key, reps, labels, params):
    
    forward_t, full_forward_t, seq_t = alpdesign.build_e2e(c)
    key, key2 = jax.random.split(key)
    start_params = seq_t.init(key, jnp.tile(
    jnp.squeeze(gen(key2, 1)), (c.model_number, 1)))
    best_v, params, train_loss, bo_loss = alpdesign.alg_iter(
        key2, reps, labels, full_forward_t, seq_t, c, x0_gen=gen,
        start_params=start_params
        )
    s = decode_seq(best_v)
    reps = np.concatenate((reps, get_reps([s])[0]))
    y = blosum(target_seq, s)
    print(s, y)
    labels = np.concatenate((labels, np.array(y).reshape(1,)))
    return key, reps, labels, s, params, bo_loss, train_loss

    

In [8]:
y = []
for i in range(30):
    print(i)
    params = None
    key, _ = jax.random.split(key, num=2)
    key, reps, labels, final_vec, params, bo_loss, mlp_loss= loop(key, reps, labels, params)
    y.append(blosum(target_seq, final_vec))

0
['L', 'V', 'F', 'M', 'H', 'G', 'G', 'R', 'G', 'G', 'G', 'G', 'H'] -0.12835352
1
['E', 'I', 'K', 'E', 'L', 'T', 'E', 'V', 'L', 'Q', 'T', 'L', 'I'] 0.049196064
2
['E', 'E', 'R', 'E', 'R', 'L', 'Q', 'K', 'R', 'E', 'Q', 'E', 'Q'] 0.51082504
3
['K', 'E', 'R', 'E', 'E', 'Q', 'R', 'R', 'R', 'E', 'E', 'E', 'R'] 0.5463349
4
['R', 'P', 'R', 'Y', 'Q', 'I', 'A', 'W', 'S', 'K', 'K', 'F', 'F'] 0.084706016
5
['L', 'Y', 'F', 'Y', 'N', 'F', 'K', 'M', 'H', 'R', 'F', 'Q', 'P'] -0.057333704
6
['K', 'E', 'R', 'E', 'R', 'E', 'R', 'Q', 'K', 'R', 'E', 'K', 'E'] 0.5463349
7
['E', 'E', 'E', 'R', 'R', 'R', 'R', 'E', 'E', 'E', 'K', 'R', 'E'] 0.43980518
8
['M', 'E', 'V', 'M', 'E', 'D', 'D', 'M', 'F', 'L', 'R', 'Y', 'L'] -0.16386345
9
['K', 'M', 'W', 'K', 'M', 'D', 'Q', 'A', 'R', 'K', 'S', 'F', 'T'] -0.16386345
10
['Q', 'F', 'D', 'F', 'L', 'H', 'L', 'W', 'M', 'D', 'V', 'E', 'Q'] 0.04919606
11
['P', 'F', 'P', 'L', 'F', 'T', 'T', 'I', 'E', 'A', 'K', 'N', 'R'] 0.084705986
12
['P', 'R', 'T', 'H', 'Q', 'L', 'N', 'I', 

In [ ]:
plt.figure()
plt.plot(y,label='y')
plt.plot(yhat, label='yhat')
plt.legend()
plt.title('Blosum Score vs N')

In [ ]:
ys_01 = ys

In [ ]:
import pickle

with open('y.pkl', 'wb') as f1:
    pickle.dump(y, f1)
    
with open('yhat.pkl', 'wb') as f2:
    pickle.dump(yhat, f2)

In [ ]:
'S', 'G', 'K', 'Y', 'Y', 'F', 'F', 'H', 'H', 'F'
'H', 'V', 'Y', 'V', 'F', 'G', 'G', 'Y', 'E', 'Y'
['GIGAVLKILIAAARPREAITAALKKR']
['GIGAVLKVLTTGLPALISWIKRKRQQ']
['H', 'K', 'D', 'I', 'I', 'K', 'A', 'E', 'G', 'V', 'K', 'E', 'G', 'E', 'P', 'I', 'G', 'P', 'C', 'I', 'N', 'N', 'S', 'Q', 'C', 'P'] 0.3318122
['A', 'G', 'D', 'P', 'I', 'K', 'C', 'V', 'R', 'V', 'G', 'E', 'T', 'I', 'P', 'E', 'G', 'S', 'P', 'L', 'M', 'P', 'C', 'M', 'M', 'T'] 0.3318122